In [49]:
import tools
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
print(os.getenv("LANGSMITH_ENDPOINT"))

https://api.smith.langchain.com


In [50]:
import os

import os
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent




llm = init_chat_model(
    "azure_openai:gpt-4.1-mini",
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    api_version="2024-12-01-preview"
)



In [ ]:
import os
from langchain_tavily import TavilySearch

web_search = TavilySearch(max_results=2)

browser_agent = create_react_agent(
    model=llm,
    tools=[web_search],
    prompt=(
        'Aufgaben:'
        '- Du bist ein Assistent, der Informationen über die Blütezeit von Pflanzen bereitstellt. Suche im Internet nach Informationen.'
        '- Antworte auf die Fragen in folgendme Format: Antworte in JSON und folge dem Schema: { "type": "object", "properties": { "response": { "type": "boolean" }, "reasoning": { "type": "string" } }, "required": [ "response", "reasoning" ] }'
    ),
    name="browser_agent",
)

In [53]:
response = browser_agent.invoke({"role": "user", "messages": "Angenommen, heute ist der 30.07.2024 in K\u00f6ln \u2013 zeigt die Kastanie schon erste Bl\u00fcten?"})

In [54]:
response["messages"][-1].content


'{\n  "response": true,\n  "reasoning": "Die Blütezeit der Kastanie liegt von Juli bis August. Da heute der 30.07.2024 ist, zeigt die Kastanie in Köln bereits erste Blüten, wie aus der Quelle mein-schoener-garten.de hervorgeht, die bestätigt, dass die weißen Blüten der Strauchkastanie von Juli bis August erscheinen."\n}'

In [58]:
import json
from utils import get_tokens
#base_prompt = 'Antworte in JSON und folge dem Schema: { "type": "object", "properties": { "response": { "type": "boolean" }, "reasoning": { "type": "string" } }, "required": [ "response", "reasoning" ] }'
with open("datasets/dataset_24.json", "r") as file:
    dataset = json.load(file)


responses = []
for data in dataset:
    sentence = data["sentence"]
    prompt = {"role": "user", "messages": sentence}

    messages = browser_agent.invoke(prompt)
    
    response = messages["messages"][-1].content
    cleaned_response = response.strip("```json").strip("```").strip()
    response_json = json.loads(cleaned_response)
    tokens = get_tokens(messages)
    responses.append({
        "id": data["id"],
        "plant": data["plant"],
        "city": data["city"],
        "date": data["date"],
        "sentence": sentence,
        "response": response_json["response"],
        "reasoning": response_json["reasoning"],
        "tokens": tokens
    })

print(responses)

with open("results/llm_with_browser_24.json", "w") as file:
    json.dump(responses, file, indent=4)



[{'id': 0, 'plant': 'Apfel', 'city': 'Berlin', 'date': '11.04.2021', 'sentence': 'Heute, am 11.04.2021, befinden wir uns in Berlin. Ist die Apfel gerade am Blühen?', 'response': True, 'reasoning': 'Laut den gefundenen Informationen beginnt die Apfelblüte in Mitteleuropa im Vollfrühling, der meist Ende März bis Anfang April einsetzt. Anfang April beginnt in Deutschland, insbesondere im Westen, die Apfelblüte. Da heute der 11.04.2021 ist und wir uns in Berlin befinden, ist es sehr wahrscheinlich, dass die Apfelbäume gerade am Blühen sind.', 'tokens': {'total_tokens': 2541, 'prompt_tokens': 2418, 'completion_tokens': 123}}, {'id': 1, 'plant': 'Apfel', 'city': 'München', 'date': '05.04.2021', 'sentence': 'Stellen wir uns vor, es ist der 05.04.2021 und wir sind in München. Blüht die Apfel jetzt?', 'response': True, 'reasoning': 'Die Apfelbäume blühen in der Regel von April bis Mai. Da heute der 05.04.2021 ist und wir uns in München befinden, ist es sehr wahrscheinlich, dass die Apfelbäume j

In [56]:
print(messages["messages"][-1].content)

{
  "response": true,
  "reasoning": "Die Birke blüht typischerweise im Frühjahr. In der Region Magdeburg ist der Blütezeitraum der Birke meist von März bis April. Daher ist es am 12.03.2020 in Magdeburg sehr wahrscheinlich, dass die Birke bereits blüht."
}
